In [2]:
import os
import pandas
import cv2
import numpy 




In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

import matplotlib.pyplot as plt
import numpy 
import pickle
import os
from imutils import paths

In [9]:
class selective_tracker():

    data = pandas.read_csv('/Users/akshitshishodia/intern/tracker/annotation/labels_my-project-name_2023-07-09-02-49-05.csv')
    pos = 0
    neg = 0
    def __init__(self) -> None:
        pass
    def calculate_iou(box_a,box_b):
        xA = max(box_a[0],box_b[0])
        yA = max(box_a[1],box_b[1])
        xB = min(box_a[2],box_b[2])
        yB = min(box_a[3],box_b[3])
        inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        pred = (box_a[2]-box_a[0]+1)*(box_a[3]-box_a[1]+1)
        ground_truth = (box_b[2]-box_b[0]+1)*(box_b[3]-box_b[1]+1)
        iou = inter_area / float(pred + ground_truth - inter_area)
        return iou
    
    @classmethod
    def annotation_extracter(cls,path):
        totalpos = 0
        totalneg = 0
        image_path = [file for file in os.listdir(path=path)]
        for image_loc in image_path:
            try:
                row = cls.data[cls.data['image_name']== image_loc]
                

                w = int(row.image_width)
                h = int(row.image_height)
                x_min = int(row.bbox_x)
                y_min = int(row.bbox_y)
                x_max= int(row.bbox_x)+int(row.bbox_width)
                y_max = int(row.bbox_y)+int(row.bbox_height)

                x_min = max(0,x_min)
                y_min = max(0,y_min)
                x_max = min(w,x_max)
                y_max = min(h,y_max)

                gt_box = (x_min,y_min,x_max,y_max)
            except:
                continue
            img = cv2.imread(os.path.join(path,image_loc))
            ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
            ss.setBaseImage(img)
            ss.switchToSelectiveSearchQuality()
            rects = ss.process()
            proposedRegions = list()
            for x,y,w,h in rects:
                proposedRegions.append((x,y,x+w,y+h))

            pos_roi = 0
            neg_roi = 0

            for proposedRect in proposedRegions[:2000]:
                (x1,y1,x2,y2) = proposedRect

                IOU = cls.calculate_iou(gt_box,proposedRect)
                roi = None
                OutputPath = None
                (gx1,gy1,gx2,gy2) = gt_box

                if IOU > 0.7 and pos_roi <= 30:
                    roi = img[y1:y2,x1:x2]
                    loc = str(totalpos)+'.png'
                    OutputPath = os.path.join('/Users/akshitshishodia/intern/tracker/refined/gripper',loc)
                    totalpos+=1
                    pos_roi+=1                            
                fullOverlap = x1 >= gx1
                fullOverlap = fullOverlap and y1 >= gy1
                fullOverlap = fullOverlap and x2 <= gx2
                fullOverlap = fullOverlap and y2 <= gy2

                if not fullOverlap and IOU < 0.1 and neg_roi <= 10:
                    roi = img[y1:y2,x1:x2]
                    loc = str(totalneg)+'.png'
                    OutputPath = os.path.join('/Users/akshitshishodia/intern/tracker/refined/not gripper',loc)
                    totalneg+=1
                    neg_roi+=1
                
                if roi is not None and OutputPath is not None:
                    roi = cv2.resize(roi,(224,224),cv2.INTER_CUBIC)
                    cv2.imwrite(OutputPath,roi)

    def build_model(self):
        EPOCHS = 5
        BS = 32
        LR = 1e-4
        data = list()
        labels = list()
        #gripper
        im_path = paths.list_images('/Users/akshitshishodia/intern/tracker/refined')

        for im in im_path:
            label = im.split(os.path.sep)[-2]
            image = load_img(im,target_size=(224,224))
            image = img_to_array(image)
            image = preprocess_input(image)
            data.append(image)
            labels.append(label)
            
        data = numpy.array(data,dtype='float32')
        labels = numpy.array(labels)


        lb = LabelBinarizer()
        labels = lb.fit_transform(labels)
        labels = to_categorical(labels)

        (trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)
        augment = ImageDataGenerator(
            rotation_range=20,
            zoom_range=0.15,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            fill_mode="nearest"
        )

        base_model = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        headModel = base_model.output
        headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
        headModel = Flatten(name="flatten")(headModel)
        headModel = Dense(128, activation="relu")(headModel)
        headModel = Dropout(0.5)(headModel)
        headModel = Dense(2, activation="softmax")(headModel)

        model = Model(inputs=base_model.input, outputs=headModel)

        # for layer in base_model.layers():
        #     layer.trainable = False
        
        opt =  Adam(LR)
        model.compile(loss = 'binary_crossentropy',optimizer = opt,metrics = ['accuracy'])
        History = model.fit(
        augment.flow(trainX, trainY, batch_size=BS),
        steps_per_epoch=len(trainX) // BS,
        validation_data=(testX, testY),
        validation_steps=len(testX) // BS,
        epochs=EPOCHS)


        model.save('model_3.h5',save_format = 'h5')
        f = open('label_encoder.pickle', "wb")
        f.write(pickle.dumps(lb))
        f.close()
        return History,model



In [10]:
tracker = selective_tracker()

history,_ = tracker.build_model()

Epoch 1/5
12/12 [==============================] - 23s 2s/step - loss: 0.4094 - accuracy: 0.8784 - val_loss: 0.2149 - val_accuracy: 0.9802
Epoch 2/5
12/12 [==============================] - 19s 2s/step - loss: 0.0991 - accuracy: 0.9865 - val_loss: 0.1657 - val_accuracy: 0.9802
Epoch 3/5
12/12 [==============================] - 21s 2s/step - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.1253 - val_accuracy: 0.9802
Epoch 4/5
12/12 [==============================] - 19s 2s/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.0941 - val_accuracy: 0.9901
Epoch 5/5
12/12 [==============================] - 19s 2s/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 0.0771 - val_accuracy: 0.9901


/Users/akshitshishodia/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
import tensorflow

import imutils
model = tensorflow.keras.models.load_model('/Users/akshitshishodia/intern/tracker/model_3.h5')
lb = pickle.loads(open('/Users/akshitshishodia/intern/tracker/label_encoder.pickle','rb').read())


TEST

In [12]:

test_image = cv2.imread('/Users/akshitshishodia/intern/tracker/DATASET_/frame220.png')

test_image = imutils.resize(test_image,width=500)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(test_image)
ss.switchToSelectiveSearchQuality()
rects = ss.process()
proposals = list()
box= list()
for x,y,w,h in rects[:200]:
    roi = test_image[y:y+h,x:x+w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi,(224,224),interpolation=cv2.INTER_CUBIC)
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    proposals.append(roi)
    box.append((x,y,x+w,y+h))
proposals = numpy.array(proposals,dtype = numpy.int32)
box = numpy.array(box,dtype = numpy.int32)

proba = model.predict(proposals)
labels = lb.classes_[numpy.argmax(proba,axis =1 )]
index = numpy.where(labels == 'gripper')[0]
box = box[index]
proba = proba[index][:,1]
index = numpy.where(proba >= 0.90)
box = box[index]
proba = proba[index]
clone = test_image.copy()
for (box, prob) in zip(box, proba):
	(startX, startY, endX, endY) = box
	cv2.rectangle(clone, (startX, startY), (endX, endY),
		(0, 255, 0), 2)
	y = startY - 10 if startY - 10 > 10 else startY + 10
	text= "gripper: {:.2f}%".format(prob * 100)
	cv2.putText(clone, text, (startX, y),
		cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
cv2.imshow("Before NMS", clone)
cv2.waitKey(0)
cv2.destroyAllWindows()




7/7 [==============================] - 2s 193ms/step


KeyboardInterrupt: 

: 